In [13]:
%env http_proxy=http://127.0.0.1:10890
%env https_proxy=http://127.0.0.1:10890
%env no_proxy=127.0.0.1,localhost
%env HTTP_PROXY=http://127.0.0.1:10890
%env HTTPS_PROXY=http://127.0.0.1:10890
%env NO_PROXY=127.0.0.1,localhost
!echo -e "\033[32m[√] 已开启代理\033[0m"

env: http_proxy=http://127.0.0.1:10890
env: https_proxy=http://127.0.0.1:10890
env: no_proxy=127.0.0.1,localhost
env: HTTP_PROXY=http://127.0.0.1:10890
env: HTTPS_PROXY=http://127.0.0.1:10890
env: NO_PROXY=127.0.0.1,localhost
[√] 已开启代理


In [14]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
)
from autogen.agentchat.contrib import agent_builder
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor
from dotenv import load_dotenv
load_dotenv()
config_list = [
    {"model": "gpt-4-1106-preview", "api_key": os.environ["OPENAI_API_KEY"]},
    {"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]},
]

In [15]:
task = (
    f"""Provide the most relevant GitHub repository about the following title and abstract: 3D Gaussian Splatting for Real-Time Radiance Field Rendering Radiance Field methods have recently revolutionized novel-view synthesis of scenes captured with multiple photos or videos. However, achieving high visual quality still requires neural networks that are costly to train and render, while recent faster methods inevitably trade off speed for quality. For unbounded and complete scenes (rather than isolated objects) and 1080p resolution rendering, no current method can achieve real-time display rates. We introduce three key elements that allow us to achieve state-of-the-art visual quality while maintaining competitive training times and importantly allow high-quality real-time (≥ 30 fps) novel-view synthesis at 1080p resolution. First, starting from sparse points produced during camera calibration, we represent the scene with 3D Gaussians that preserve desirable properties of continuous volumetric radiance fields for scene optimization while avoiding unnecessary computation in empty space; Second, we perform interleaved optimization/density control of the 3D Gaussians, notably optimizing anisotropic covariance to achieve an accurate representation of the scene; Third, we develop a fast visibility-aware rendering algorithm that supports anisotropic splatting and both accelerates training and allows realtime rendering. We demonstrate state-of-the-art visual quality and real-time rendering on several established datasets.
        requirements: code written in C kernel and can run on the CPU
    """
)
print(task)

Provide the most relevant GitHub repository about the following title and abstract: 3D Gaussian Splatting for Real-Time Radiance Field Rendering Radiance Field methods have recently revolutionized novel-view synthesis of scenes captured with multiple photos or videos. However, achieving high visual quality still requires neural networks that are costly to train and render, while recent faster methods inevitably trade off speed for quality. For unbounded and complete scenes (rather than isolated objects) and 1080p resolution rendering, no current method can achieve real-time display rates. We introduce three key elements that allow us to achieve state-of-the-art visual quality while maintaining competitive training times and importantly allow high-quality real-time (≥ 30 fps) novel-view synthesis at 1080p resolution. First, starting from sparse points produced during camera calibration, we represent the scene with 3D Gaussians that preserve desirable properties of continuous volumetric 

In [16]:
import os
from autogen import ConversableAgent, register_function
from typing import Annotated, Literal
from typing import List, Dict, Tuple

from assistant import check_local
from search import search_github
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from search import search_github
from download import load_readme, clone_github_repo
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
embeddings = OpenAIEmbeddings()
def download_readme_to_db(keywords: List[str]) -> Annotated[str, "path of vector database"]:
    keywords = [keyword.lower().strip() for keyword in keywords][:1]
    keywords = ["gaussian splatting"]
    db_path = check_local(keywords)
    # vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    if not db_path:
        repos = search_github(keywords, 1)
        vector_store = FAISS(
                        embedding_function=embeddings,
                        index=faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
                        docstore=InMemoryDocstore(),
                        index_to_docstore_id={},
                    )
        load_readme(repos, vector_store)
        db_file_name = '-'.join(keywords).lower()
        db_path = f'./db/{db_file_name}'
        vector_store.save_local(db_path)
    return db_path

def search_db(db_path: str, text: str) -> List[str]:
    vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    bm25_retriever = BM25Retriever.from_documents(vector_store.docstore._dict.values())
    faiss_retriever = vector_store.as_retriever()
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever], weights=[0.8, 0.2]
    )
    similar_repos = ensemble_retriever.invoke(text)
    return [{"repo_name": repo.metadata["repo_name"], "repo_readme": repo.page_content} for repo in similar_repos]

In [17]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Give the task.",
    code_execution_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

reviewer = AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Reviewer. Review the top related repos provided by Tool_user after use search_db and select the one that best matches the user's algorithm requirements. 
    After returning the repository name, output TERMINATE to signal the completion of the task.
    """,
)

searcher = AssistantAgent(
    name="Tool_user",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Tool_user. You need to plan how to use the tools you have. 
    Capable of understanding a detailed algorithm description provided by the user, 
    interpreting the algorithm code needed, and determining the requirements for the code (such as the programming language, whether it can run on CPU/GPU, etc.). 
    You will find the GitHub repository that best meets these requirements. 
    1. You should use download_readme_to_db to get the path of db before search the database.
    2. Use the text generated by Writer to seach the database.
    """,
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""Writer. Your task is to generate a README file for a project that aligns as closely as possible with the user's specified algorithm requirements. 
    The README must highlight and prioritize the user's specific needs, such as programming language, hardware compatibility (CPU/GPU), performance optimization, and any other explicit details provided in the algorithm description.
    """,
)

groupchat = GroupChat(
    agents=[user_proxy, tool_user, reviewer, writer],
    messages=[],
    max_round=15,
    speaker_selection_method="auto",
)
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list, "cache_seed": None})

register_function(
    search_db,
    caller=tool_user,
    executor=user_proxy,
    description="Search with the text from a vector db loaded from db_path, and return the descriptions and readme of top related repos"
)
register_function(
    download_readme_to_db,
    caller=tool_user,
    executor=user_proxy,
    description="Search github repos with a list of detailed keywords, and save the readme of them into a vector database. Return the path of db"
)

In [18]:
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache,
    )

Admin (to chat_manager):

Provide the most relevant GitHub repository about the following title and abstract: 3D Gaussian Splatting for Real-Time Radiance Field Rendering Radiance Field methods have recently revolutionized novel-view synthesis of scenes captured with multiple photos or videos. However, achieving high visual quality still requires neural networks that are costly to train and render, while recent faster methods inevitably trade off speed for quality. For unbounded and complete scenes (rather than isolated objects) and 1080p resolution rendering, no current method can achieve real-time display rates. We introduce three key elements that allow us to achieve state-of-the-art visual quality while maintaining competitive training times and importantly allow high-quality real-time (≥ 30 fps) novel-view synthesis at 1080p resolution. First, starting from sparse points produced during camera calibration, we represent the scene with 3D Gaussians that preserve desirable properties


Next speaker: Tool_user

Tool_user (to chat_manager):

***** Suggested tool call (call_nSFCLFGcm1ki6ZLvoMBS6utf): download_readme_to_db *****
Arguments: 
{"keywords":["3D Gaussian Splatting","Radiance Field Rendering","novel-view synthesis","real-time display","anisotropic covariance","visibility-aware rendering algorithm"]}
**************************************************************************************

--------------------------------------------------------------------------------

Next speaker: Admin


>>>>>>>> EXECUTING FUNCTION download_readme_to_db...
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling tool (call_nSFCLFGcm1ki6ZLvoMBS6utf) *****
./db/gaussian splatting
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: Writer

Writer (to chat_manager):

### 3D Gaussian Splatting for Real-Time Radiance Field Rendering README

####